In [1]:
import pandas as pd
from datasets import load_dataset
import re

In [2]:
dataset = load_dataset("fancyzhx/yelp_polarity")

README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [3]:
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

In [4]:
train_df.head()

,text,label
0,"Unfortunately, the frustration of being Dr. Go...",0
1,Been going to Dr. Goldberg for over 10 years. ...,1
2,I don't know what Dr. Goldberg was like before...,0
3,I'm writing this review to give you a heads up...,0
4,All the food is great here. But the best thing...,1


In [5]:
test_df.head()

,text,label
0,"Contrary to other reviews, I have zero complai...",1
1,Last summer I had an appointment to get new ti...,0
2,"Friendly staff, same starbucks fair you get an...",1
3,The food is good. Unfortunately the service is...,0
4,Even when we didn't have a car Filene's Baseme...,1


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    560000 non-null  object
 1   label   560000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.5+ MB


In [7]:
train_df.isnull().sum()

text     0
label    0
dtype: int64

In [8]:
train_df.sample(5)['text'].values

array(["More of a Starbucks walk-through. Close to the pool, so you can hop on over for your favorite drink. They only have 3 tables that are usually being used by people waiting for their taxi to pick them up from the hotel. I'm going to assume that having 1 row of slot machines is more important than having a waiting/seating area for guests. The 500+ other slot machines they have in the casino area is not enough. If you just need your coffee fix, great place. If you need a quiet place to get work done, not the spot. \\n\\nWHAT SUCKS: They do not take Starbucks cards, and they allow smoking in the area. I will never get any food products from a location that allows smoking in the same area. I'm going to start not giving my business to ANY location that lets people smoke inside. When is Las Vegas going to hop on baord with NYC and other cities that outlaw smoking in certain establishments/areas. Vegas is supposed to be forward thinking, keeping up with everything new. Nope, they are st

In [9]:
def clean_text(text):
    text = text.replace('\\n', ' ') 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\\"', '"')
    text = text.strip()     
    return text
train_df['text'] = train_df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)

In [10]:
train_df.duplicated(subset='text').sum()

49

In [11]:
train_df = train_df.drop_duplicates(subset='text').reset_index(drop=True)

In [12]:
train_df.duplicated(subset='text').sum()

0

In [13]:
test_df.duplicated(subset='text').sum()

0

In [14]:
from sklearn.model_selection import train_test_split
from datasets import Dataset


In [15]:

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [16]:

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = dataset['test']